# For MSD dataset

In [ ]:
import os
import json
import h5py
import pandas as pd
import sqlite3
import sys

In [ ]:
dataset_path = '/Users/vivekrambha/Documents/QMUL Slides and Notes/Music recommendation system/millionsongsubset/MillionSongSubset'

In [ ]:
def load_song_features(file_path):
    with h5py.File(file_path, 'r') as f:
        features = {
            'segments_pitches': f['/analysis/segments_pitches'][:].tolist()
        }
    return features

In [ ]:
def load_metadata(file_path):
    with h5py.File(file_path, 'r') as f:
        artist_name = f['/metadata/songs'][0]['artist_name'].decode('utf-8')
        song_title = f['/metadata/songs'][0]['title'].decode('utf-8')
        album_name = f['/metadata/songs'][0]['release'].decode('utf-8')
    return artist_name, song_title, album_name

In [ ]:
def extract_all_features(dataset_path):
    data = []
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.endswith('.h5'):
                file_path = os.path.join(root, file)
                try:
                    features = load_song_features(file_path)
                    artist_name, song_title, album_name = load_metadata(file_path)
                    song_id = os.path.splitext(file)[0]  # Using file name as song ID
                    data.append({
                        'song_id': song_id,
                        'segments_pitches': json.dumps(features['segments_pitches']),
                        'artist_name': artist_name,
                        'song_title': song_title,
                        'album_name': album_name
                    })
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
    return data

In [ ]:
all_song_data = extract_all_features(dataset_path)

df = pd.DataFrame(all_song_data)

In [ ]:
# Save data to CSV
df.to_csv('song_features.csv', index=False)
print("Features and metadata saved to song_features.csv")

# For MXM dataset

In [ ]:
#!/usr/bin/env python3
"""
Thierry Bertin-Mahieux (2011) Columbia University
tb2332@columbia.edu

This code puts the musiXmatch dataset (format: 2 text files)
into a SQLite database for ease of use.

This is part of the Million Song Dataset project from
LabROSA (Columbia University) and The Echo Nest.
http://labrosa.ee.columbia.edu/millionsong/

Copyright 2011, Thierry Bertin-Mahieux

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
"""

import os
import sys
import sqlite3


def encode_string(s):
    """
    Simple utility function to make sure a string is proper
    to be used in a SQLite query
    (different than posgtresql, no N to specify unicode)
    EXAMPLE:
      That's my boy! -> 'That''s my boy!'
    """
    return "'" + s.replace("'", "''") + "'"


def die_with_usage():
    """ HELP MENU """
    print('mxm_dataset_to_db.py')
    print('   by T. Bertin-Mahieux (2011) Columbia University')
    print('      tb2332@columbia.edu')
    print('This code puts the musiXmatch dataset into an SQLite database.')
    print('')
    print('USAGE:')
    print('  ./mxm_dataset_to_db.py <train> <test> <output.db>')
    print('PARAMS:')
    print('      <train>  - mXm dataset text train file')
    print('       <test>  - mXm dataset text test file')
    print('  <output.db>  - SQLite database to create')
    sys.exit(0)


if __name__ == '__main__':

    # help menu
    if len(sys.argv) < 4:
        die_with_usage()

    # params
    trainf = sys.argv[1]
    testf = sys.argv[2]
    outputf = sys.argv[3]

    # sanity checks
    if not os.path.isfile(trainf):
        print(f'ERROR: {trainf} does not exist.')
        sys.exit(0)
    if not os.path.isfile(testf):
        print(f'ERROR: {testf} does not exist.')
        sys.exit(0)
    if os.path.exists(outputf):
        print(f'ERROR: {outputf} already exists.')
        sys.exit(0)

    # open output SQLite file
    conn = sqlite3.connect(outputf)

    # create tables -> words and lyrics
    q = "CREATE TABLE words (word TEXT PRIMARY KEY)"
    conn.execute(q)
    q = """
    CREATE TABLE lyrics (
        track_id TEXT,
        mxm_tid INT,
        word TEXT,
        count INT,
        is_test INT,
        FOREIGN KEY(word) REFERENCES words(word)
    )
    """
    conn.execute(q)

    # get words, put them in the words table
    with open(trainf, 'r') as f:
        for line in f:
            if line == '':
                continue
            if line[0] == '%':
                topwords = line.strip()[1:].split(',')
                break
    for w in topwords:
        q = f"INSERT INTO words VALUES({encode_string(w)})"
        conn.execute(q)
    conn.commit()
    # sanity check, make sure the words were entered according
    # to popularity, most popular word should have ROWID 1
    q = "SELECT ROWID, word FROM words ORDER BY ROWID"
    res = conn.execute(q)
    tmpwords = res.fetchall()
    assert len(tmpwords) == len(topwords), 'Number of words issue.'
    for k in range(len(tmpwords)):
        assert tmpwords[k][0] == k + 1, 'ROWID issue.'
        assert tmpwords[k][1] == topwords[k], 'ROWID issue.'
    print("'words' table filled, checked.")

    # we put the train data in the dataset
    with open(trainf, 'r') as f:
        cnt_lines = 0
        for line in f:
            if line == '' or line.strip() == '':
                continue
            if line[0] in ('#', '%'):
                continue
            lineparts = line.strip().split(',')
            tid = lineparts[0]
            mxm_tid = lineparts[1]
            for wordcnt in lineparts[2:]:
                wordid, cnt = wordcnt.split(':')
                q = f"""
                INSERT INTO lyrics
                SELECT '{tid}', {mxm_tid}, words.word, {cnt}, 0
                FROM words WHERE words.ROWID={wordid}
                """
                conn.execute(q)
            # verbose
            cnt_lines += 1
            if cnt_lines % 15000 == 0:
                print(f'Done with {cnt_lines} train tracks.')
                conn.commit()
    conn.commit()
    print('Train lyrics added.')

    # we put the test data in the dataset
    # only difference from train: is_test is now 1
    with open(testf, 'r') as f:
        cnt_lines = 0
        for line in f:
            if line == '' or line.strip() == '':
                continue
            if line[0] in ('#', '%'):
                continue
            lineparts = line.strip().split(',')
            tid = lineparts[0]
            mxm_tid = lineparts[1]
            for wordcnt in lineparts[2:]:
                wordid, cnt = wordcnt.split(':')
                q = f"""
                INSERT INTO lyrics
                SELECT '{tid}', {mxm_tid}, words.word, {cnt}, 1
                FROM words WHERE words.ROWID={wordid}
                """
                conn.execute(q)
            # verbose
            cnt_lines += 1
            if cnt_lines % 15000 == 0:
                print(f'Done with {cnt_lines} test tracks.')
                conn.commit()
    conn.commit()
    print('Test lyrics added.')

    # create indices
    q = "CREATE INDEX idx_lyrics1 ON lyrics (track_id)"
    conn.execute(q)
    q = "CREATE INDEX idx_lyrics2 ON lyrics (mxm_tid)"
    conn.execute(q)
    q = "CREATE INDEX idx_lyrics3 ON lyrics (word)"
    conn.execute(q)
    q = "CREATE INDEX idx_lyrics4 ON lyrics (count)"
    conn.execute(q)
    q = "CREATE INDEX idx_lyrics5 ON lyrics (is_test)"
    conn.execute(q)
    conn.commit()
    print('Indices created.')

    # close output SQLite connection
    conn.close()


In [ ]:
def encode_string(s):
    """
    Simple utility function to make sure a string is proper
    to be used in a SQLite query
    (different than posgtresql, no N to specify unicode)
    EXAMPLE:
      That's my boy! -> 'That''s my boy!'
    """
    return "'" + s.replace("'", "''") + "'"

def die_with_usage():
    """ HELP MENU """
    print('mxm_dataset_to_db.py')
    print('   by T. Bertin-Mahieux (2011) Columbia University')
    print('      tb2332@columbia.edu')
    print('This code puts the musiXmatch dataset into an SQLite database.')
    print('')
    print('USAGE:')
    print('  ./mxm_dataset_to_db.py <train> <test> <output.db>')
    print('PARAMS:')
    print('      <train>  - mXm dataset text train file')
    print('       <test>  - mXm dataset text test file')
    print('  <output.db>  - SQLite database to create')
    sys.exit(0)


In [ ]:
trainf = "/Users/vivekrambha/Documents/QMUL Slides and Notes/Music recommendation system/mxm_dataset_train.txt"
testf = "/Users/vivekrambha/Documents/QMUL Slides and Notes/Music recommendation system/mxm_dataset_test.txt"
outputf = "music_recommendation.db"

In [ ]:
if not os.path.isfile(trainf):
    print(f'ERROR: {trainf} does not exist.')
    die_with_usage()
if not os.path.isfile(testf):
    print(f'ERROR: {testf} does not exist.')
    die_with_usage()
if os.path.exists(outputf):
    print(f'ERROR: {outputf} already exists.')
    die_with_usage()

In [ ]:
conn = sqlite3.connect(outputf)

q = "CREATE TABLE words (word TEXT PRIMARY KEY)"
conn.execute(q)
q = """
CREATE TABLE lyrics (
    track_id TEXT,
    mxm_tid INT,
    word TEXT,
    count INT,
    is_test INT,
    FOREIGN KEY(word) REFERENCES words(word)
)
"""
conn.execute(q)

In [ ]:
with open(trainf, 'r') as f:
    for line in f:
        if line == '':
            continue
        if line[0] == '%':
            topwords = line.strip()[1:].split(',')
            break
for w in topwords:
    q = f"INSERT INTO words VALUES({encode_string(w)})"
    conn.execute(q)
conn.commit()

q = "SELECT ROWID, word FROM words ORDER BY ROWID"
res = conn.execute(q)
tmpwords = res.fetchall()
assert len(tmpwords) == len(topwords), 'Number of words issue.'
for k in range(len(tmpwords)):
    assert tmpwords[k][0] == k + 1, 'ROWID issue.'
    assert tmpwords[k][1] == topwords[k], 'ROWID issue.'
print("'words' table filled, checked.")

with open(trainf, 'r') as f:
    cnt_lines = 0
    for line in f:
        if line == '' or line.strip() == '':
            continue
        if line[0] in ('#', '%'):
            continue
        lineparts = line.strip().split(',')
        tid = lineparts[0]
        mxm_tid = lineparts[1]
        for wordcnt in lineparts[2:]:
            wordid, cnt = wordcnt.split(':')
            q = f"""
            INSERT INTO lyrics
            SELECT '{tid}', {mxm_tid}, words.word, {cnt}, 0
            FROM words WHERE words.ROWID={wordid}
            """
            conn.execute(q)
        cnt_lines += 1
        if cnt_lines % 15000 == 0:
            print(f'Done with {cnt_lines} train tracks.')
            conn.commit()
conn.commit()
print('Train lyrics added.')


In [ ]:
with open(testf, 'r') as f:
    cnt_lines = 0
    for line in f:
        if line == '' or line.strip() == '':
            continue
        if line[0] in ('#', '%'):
            continue
        lineparts = line.strip().split(',')
        tid = lineparts[0]
        mxm_tid = lineparts[1]
        for wordcnt in lineparts[2:]:
            wordid, cnt = wordcnt.split(':')
            q = f"""
            INSERT INTO lyrics
            SELECT '{tid}', {mxm_tid}, words.word, {cnt}, 1
            FROM words WHERE words.ROWID={wordid}
            """
            conn.execute(q)
        cnt_lines += 1
        if cnt_lines % 15000 == 0:
            print(f'Done with {cnt_lines} test tracks.')
            conn.commit()
conn.commit()
print('Test lyrics added.')

In [ ]:
q = "CREATE INDEX idx_lyrics1 ON lyrics (track_id)"
conn.execute(q)
q = "CREATE INDEX idx_lyrics2 ON lyrics (mxm_tid)"
conn.execute(q)
q = "CREATE INDEX idx_lyrics3 ON lyrics (word)"
conn.execute(q)
q = "CREATE INDEX idx_lyrics4 ON lyrics (count)"
conn.execute(q)
q = "CREATE INDEX idx_lyrics5 ON lyrics (is_test)"
conn.execute(q)
conn.commit()
print('Indices created.')

In [ ]:
conn.close()
print('Database connection closed.')

In [ ]:
db_path = "/Users/vivekrambha/Documents/QMUL Slides and Notes/Music recommendation system/music_recommendation.db"
conn = sqlite3.connect(db_path)

In [ ]:
# Query data from the 'words' table
words_query = "SELECT * FROM words"
words_df = pd.read_sql_query(words_query, conn)

# Query data from the 'lyrics' table
lyrics_query = "SELECT * FROM lyrics"
lyrics_df = pd.read_sql_query(lyrics_query, conn)


In [ ]:
# Save the 'words' table data to CSV
words_csv_path = "words.csv"
words_df.to_csv(words_csv_path, index=False)

# Save the 'lyrics' table data to CSV
lyrics_csv_path = "lyrics.csv"
lyrics_df.to_csv(lyrics_csv_path, index=False)

print("CSV files created:")
print(f"- {words_csv_path}")
print(f"- {lyrics_csv_path}")

In [ ]:
# Load the CSV files
df1 = pd.read_csv('/Users/vivekrambha/Documents/QMUL Slides and Notes/Music recommendation system/song_features.csv')
df2 = pd.read_csv('/Users/vivekrambha/Documents/QMUL Slides and Notes/Music recommendation system/lyrics.csv')
df3 = pd.read_csv('/Users/vivekrambha/Documents/QMUL Slides and Notes/Music recommendation system/words.csv')


In [ ]:
# Merge df1 and df2 on song_id (track_id) and track_id
merged_df = pd.merge(df1, df2, left_on='song_id', right_on='track_id')

# Merge the resulting DataFrame with df3 on word
final_df = pd.merge(merged_df, df3, on='word')

In [ ]:
output_path = 'combined_data.csv'
final_df.to_csv(output_path, index=False)
print(f"Combined CSV file saved to {output_path}")